<a href="https://colab.research.google.com/github/gusper01/Ejercicio_ML/blob/main/Notebooks/ML_Prediccion_TasaInflacionUSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de MACHINE LEARNING para predecir Tasa de Inflación en USA

Se plantea como problema a resolver: La predicción de la tasa de inflación de USA utilizando datos/variables macroeconómicas. En general este problema se aborda combinando  técnicas de regresión y  series de tiempo.

Este caso se trata de series de tiempo con indicadores macroeconómicos publicadas por la Reserva Federal (USA FED)

|Fecha  | Inflación| CPIAUCSL | UNRATE | FEDFUNDS  | GDPC1 |
|---:|-------------:|---------:|------:|------:|------:|
|01/08/2016 | -?-  | 240.545   | 4.9   | 0.40     | -----|
|01/09/2016 | -?-  | 241.176   | 5.0   | 0.40     | -----|
|01/10/2016 | -?-  | 241.741   | 4.9   | 0.40     | 17812.56|
|01/11/2016 | -?-  | 242.026   | 4.7   | 0.41     | -----|   

Para este tipo de problemas se pueden aplicar
modelos estadísticos o econométricos como por ejemplo ARIMA o modelos de machine learning de árboles de decisión, en este caso Random Forest y XGBOOST. También se podrian aplicar modelos de redes neuronales como por ejemplo redes neuronales recurrentes (RNN) pueden ser útiles para modelar la dependencia temporal de las variables. Una opción es utilizar una arquitectura LSTM (Long Short-Term Memory), que es capaz de aprender patrones de larga duración en series de tiempo. Para este ejericio se opto por utilizar Random Forest, XGBOOST y Linear Regression.




Tenemos un conjunto de datos de series de tiempo que contiene los siguientes indicadores macroeconómicos de USA. IPC (Indice de Precios al Consumidor) PBI (Producto Bruto Interno) , Tasa de desempleo y Tasa de interés. Cada indicador se mide mensualmente. En el caso del PBI (Serie GDPC1)  se publica trimestralmente por lo que se deberá decidir si se eliminan los observaciones con datos faltantes o se imputan valores o se descarta la variable.  

Estructura del conjunto de datos:

*   Fecha (DD/MM/AAAA)
*   Indice de Precios Consumidor (en porcentaje)
*   Tasa de interés (en porcentaje)
*   Tasa de desempleo (en porcentaje)
*   PBI (en miles de millones de dólares)



#Librerias

In [1]:
!pip install fred
!pip install fredapi


  Preparing metadata (setup.py) ... done
  Created wheel for fred: filename=fred-3.1-py3-none-any.whl size=3895 sha256=06014a84be222dc4efca30ddbd8f2e308ad56dbaca29e42bbe5502a32acd5925
  Stored in directory: /root/.cache/pip/wheels/ff/66/4e/52148d2f8680f6fbc96635c13272981a9b058af93bda192e28
Successfully built fred


In [2]:
#Estadísticas
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

#MACHINE LEARNING
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
#librerias de metricas ML para regresion
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import TimeSeriesSplit
#Otras
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import seaborn as sns
from fredapi import Fred #librerias para API de FRED
import random
import pickle

# Paso 1: Obtener los datos

Para este ejercicio utlizamos las siguientes series de tiempo:
*   "CPIAUCSL" este el índice de precios al
consumidor para todas las áreas urbanas de Estados Unidos y representa el cambio porcentual mensual de los precios de una canasta de bienes y servicios consumidos por los hogares urbanos.
*   "FEDFUNDS"  tasa de interes
*   "UNRATE"  Ind. de desempleo
*   "IPMAN"   Ind. Prod. Ind. Manufacturas  
*   "MYAGM2USM052S"  M2 Dinero circulante en USA
     https://fred.stlouisfed.org/series/MYAGM2USM052S
*   "MICHM"  Expectativa de Inflación
*   "UMCSENT" Sentimiento de Inflación
*   "MARTSMPCSM44000USS" Porcentaje Ventas Minoristas Mensuales
     https://fred.stlouisfed.org/series/MARTSMPCSM44000USS
*   "WPUSI012011" Ind. Precios al Productor(PPI) Materiales de Construcción
     https://fred.stlouisfed.org/series/WPUSI012011


Publicado en [FRED](https://fred.stlouisfed.org/docs/api/fred/overview.html) sitio de la Reserva Federal de Estados Unidos.

In [3]:
# Conectar con la API de FRED y obtener los datos
np.random.seed(102324)
fred = Fred(api_key='Se debe generar API key en el sitio de FRED')
data2 = pd.DataFrame()
#data3 = pd.DataFrame()
#data4 = pd.DataFrame()
data5 = pd.DataFrame()
data2['CPI']= fred.get_series('CPIAUCSL')
data2 = data2.sort_index()
data2['Inflacion'] = ((data2['CPI'] - data2['CPI'].shift(1)) / (data2['CPI'].shift(1))) * 100
data2['Tasa InteresP'] = fred.get_series('FEDFUNDS')
data2['Tasa DesempleoP'] = fred.get_series('UNRATE')
data2['Expec. Inflacion'] = fred.get_series('MICH')
data2['Senti. Inflacion'] = fred.get_series('UMCSENT')
data2['Indice Ind. Manuf'] = fred.get_series('IPMAN')
data2['Ventas Minoristas Mensuales'] = fred.get_series('MARTSMPCSM44000USS')
data2['PPI Mat. Construccion'] = fred.get_series('WPUSI012011')
#data2['Indice Prod. Industrial'] = fred.get_series('INDPRO')
#Se elimina INDPRO por demasiado coeficiente de correlacion con otra
data5['USD Circulante'] = fred.get_series('MYAGM2USM052S')


# Se elimina serie de tiempo de PBI de FRED
# para evitar complejidad en el manejo de datos
# la serie de tiempo es trimestral y se deberia transformar a mensual e imputar
# valores faltantes o transformar las series mensuales a trimestrales

#data3['PBI'] = fred.get_series('GDPC1')
#data2['PBI'] = fred.get_series('GDPC1')

## resampleo de q a m
##data4 = data4.sort_index()
##data4 = data3.resample('M').bfill()
##data4 = data4.resample('M', loffset='D').first()
## fin
#data2['PBI'] = data4['PBI'].shift(1)

# Se agregan otras variables macroeconómicas para generar Dataset para el Modelo de Machine Learning
data2['Tasa Interes'] = data2['Tasa InteresP'].shift(1)
data2['Tasa Desempleo'] = data2['Tasa DesempleoP'].shift(1)
data2['Tasa USD Circulante']  = ((data5['USD Circulante'] - data5['USD Circulante'].shift(1)) / (data5['USD Circulante'].shift(1))) * 100

#variable predictora
#Tasa de inflación = ((CPIAUCSL actual - CPIAUCSL del mes anterior) / CPIAUCSL del mes anterior) * 100
#se calcula la tasa de inflación anual utilizando la función shift(1)
#para desplazar el índice 1 mes hacia atrás y así poder calcular la variación porcentual anual

data2 = data2.drop(['Tasa InteresP', 'Tasa DesempleoP'], axis=1)
data2.index = pd.to_datetime(data2.index)
data2.sort_index(inplace=True)

# Paso 2: Análisis Exploratorio

In [4]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 918 entries, 1947-01-01 to 2023-06-01
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CPI                          918 non-null    float64
 1   Inflacion                    917 non-null    float64
 2   Expec. Inflacion             545 non-null    float64
 3   Senti. Inflacion             637 non-null    float64
 4   Indice Ind. Manuf            617 non-null    float64
 5   Ventas Minoristas Mensuales  376 non-null    float64
 6   PPI Mat. Construccion        917 non-null    float64
 7   Tasa Interes                 827 non-null    float64
 8   Tasa Desempleo               905 non-null    float64
 9   Tasa USD Circulante          698 non-null    float64
dtypes: float64(10)
memory usage: 78.9 KB


In [5]:
data2

,CPI,Inflacion,Expec. Inflacion,Senti. Inflacion,Indice Ind. Manuf,Ventas Minoristas Mensuales,PPI Mat. Construccion,Tasa Interes,Tasa Desempleo,Tasa USD Circulante
1947-01-01,21.480,NaN,NaN,NaN,NaN,NaN,22.200,NaN,NaN,NaN
1947-02-01,21.620,0.651769,NaN,NaN,NaN,NaN,22.500,NaN,NaN,NaN
1947-03-01,22.000,1.757632,NaN,NaN,NaN,NaN,22.900,NaN,NaN,NaN
1947-04-01,22.000,0.000000,NaN,NaN,NaN,NaN,23.200,NaN,NaN,NaN
1947-05-01,21.950,-0.227273,NaN,NaN,NaN,NaN,23.300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-02-01,301.648,0.370006,4.1,67.0,100.1772,-0.4,330.470,4.33,3.4,NaN
2023-03-01,301.808,0.053042,3.6,62.0,99.5000,-1.1,331.714,4.57,3.6,NaN
2023-04-01,302.918,0.367783,4.6,63.5,100.4791,0.4,333.488,4.65,3.5,NaN
2023-05-01,303.294,0.124126,4.2,59.2,100.5996,0.3,337.553,4.83,3.4,NaN


En la visualizacion preliminiar de los datos se pueden observar que existen datos faltantes para algunos periodos y por la perioricidad trimestral del PBI. Se cuentan con 917 observcaciones, desde el 1947-01-01 a 2023-05-01

## CPIAUCSL:
1947-07-01 to 2023-05-01
Units: Index 1982-1984=100, Seasonally Adjusted
Frequency: Monthly
he Consumer Price Index for All Urban Consumers: All Items (CPIAUCSL) is a price index of a basket of goods and services paid by urban consumers.` Percent changes in the price index measure the inflation rate between any two time periods. The most common inflation metric is the percent change from one year ago`. It can also represent the buying habits of urban consumers. This particular index includes roughly 88 percent of the total population, accounting for wage earners, clerical workers, technical workers, self-employed, short-term workers, unemployed, retirees, and those not in the labor force.
The CPIs are based on prices for food, clothing, shelter, and fuels; transportation fares; service fees (e.g., water and sewer service); and sales taxes. Prices are collected monthly.
The CPI can be used to recognize periods of inflation and deflation. Significant increases in the CPI within a short time frame might indicate a period of inflation, and significant decreases in CPI within a short time frame might indicate a period of deflation. However, because the CPI includes volatile food and oil prices, it might not be a reliable measure of inflationary and deflationary periods.

https://fred.stlouisfed.org/series/CPIAUCSL


In [35]:
# VISUALIZACIONES
#Plotear la serie de datos
# Plotly IPC
fig = go.Figure()
# Agregar la serie de tiempo al gráfico
fig.add_trace(go.Scatter(x=data2.index, y=data2['CPI'], mode='lines', name='Indice IPC'))
fig.update_layout(
    title='USA IPC -Time Series FRED',
    xaxis_title='Período',
    yaxis_title='Indice IPC',
    #yaxis=dict(tickformat='.0%'),
    showlegend=True
)
# Mostrar el gráfico
fig.show()

## FEDFUNDS:

1954-07-01 to 2023-05-01
Units:  Percent, Not Seasonally Adjusted
Frequency:  Monthly
The federal funds rate is the interest rate at which depository institutions trade federal funds (balances held at Federal Reserve Banks) with each other overnight. When a depository institution has surplus balances in its reserve account, it lends to other banks in need of larger balances. In simpler terms, a bank with excess cash, which is often referred to as liquidity, will lend to another bank that needs to quickly raise liquidity.
https://fred.stlouisfed.org/series/FEDFUNDS


In [ ]:
# Plotly Tasa de Interes
fig = go.Figure()
# Agregar la serie de tiempo al gráfico
fig.add_trace(go.Scatter(x=data2.index, y=data2['Tasa Interes'], mode='lines', name='Tasa de Interes'))
fig.update_layout(
    title='USA Tasa de Interes -Time Series FRED',
    xaxis_title='Periodo',
    yaxis_title='Tasa de Interes',
    showlegend=True
)
# Mostrar el gráfico
fig.show()

## UNRATE:
1948-01-01 to 2023-05-01
Units:  Percent, Seasonally Adjusted
Frequency:  Monthly
The unemployment rate represents the number of unemployed as a percentage of the labor force. Labor force data are restricted to people 16 years of age and older, who currently reside in 1 of the 50 states or the District of Columbia, who do not reside in institutions (e.g., penal and mental facilities, homes for the aged), and who are not on active duty in the Armed Forces.
This rate is also defined as the U-3 measure of labor underutilization.
The series comes from the 'Current Population Survey (Household Survey)'
https://fred.stlouisfed.org/series/UNRATE


In [ ]:
# Plotly Desempleo
fig = go.Figure()
# Agregar la serie de tiempo al gráfico
fig.add_trace(go.Scatter(x=data2.index, y=data2['Tasa Desempleo'], mode='lines', name='Tasa de Desempleo'))
fig.update_layout(
    title='USA Tasa de Desempleo - Time Series FRED',
    xaxis_title='Período',
    yaxis_title='Tasa de Desempleo',
    showlegend=True
)

# Mostrar el gráfico
fig.show()

## GDPC1:
1947-01-01 to 2023-01-01
Units:  Billions of Chained 2012 Dollars, Seasonally Adjusted Annual Rate
Frequency:  Quarterly

BEA Account Code: A191RX
Real gross domestic product is the inflation adjusted value of the goods and services produced by labor and property located in the United States.For more information see the Guide to the National Income and Product Accounts of the United States (NIPA). For more information, please visit the Bureau of Economic Analysis.

https://fred.stlouisfed.org/series/GDPC1


In [ ]:
#random.seed(123)
# Plotly PBI
data4plot = data3['PBI'].dropna() #Elimino valores nulos o NAN
fig1 = go.Figure()
# Agregar la serie de tiempo al gráfico
fig1.add_trace(go.Scatter(x=data4plot.index, y=data4plot, mode='lines', name='PBI'))
fig1.update_layout(
    title='USA PBI Time Series FRED',
    xaxis_title='Período',
    yaxis_title='PBI',
    showlegend=True
)

# Mostrar el gráfico
fig1.show()

## M2:
1959-01-01 to 2023-03-01
Units: Dollars, Seasonally Adjusted
Frequency:  Monthly

M2 comprises M1 plus (1) savings deposits (including money market deposit accounts); (2) small-denomination time deposits (time deposits in amounts of less than $100,000), less IRA and Keogh balances at other depository corporations; and (3) balances in retail money market mutual funds, less IRA and Keogh balances at money market mutual funds. Seasonally adjusted M2 is constructed by summing savings deposits, small-denomination time deposits, and retail money funds, each seasonally adjusted separately, and adding this result to seasonally adjusted M1.


https://fred.stlouisfed.org/series/MYAGM2USM052S


In [ ]:
# Plotly USD
fig = go.Figure()
# Agregar la serie de tiempo al gráfico
fig.add_trace(go.Scatter(x=data5.index, y=data5['USD Circulante'], mode='lines', name='USD Circulante'))
fig.update_layout(
    title='USD Circulante en USA',
    xaxis_title='Período',
    yaxis_title='USD Circulante en USA',
    showlegend=True
)

# Mostrar el gráfico
fig.show()


In [36]:
# Plotly Inflacion
fig = go.Figure()
# Agregar la serie de tiempo al gráfico
fig.add_trace(go.Scatter(x=data2.index, y=data2['Inflacion'], mode='lines', name='Inflacion'))
fig.update_layout(
    title='USA Tasa de Inflación',
    xaxis_title='Período',
    yaxis_title='Tasa de Inflación USA',
    showlegend=True
)

# Mostrar el gráfico
fig.show()



#Paso 3: Transformación de Datos

## Análisis de Series  de Tiempo

###Serie Estacionaria
Esta característica es fundamental para el análisis y modelado de series de tiempo. Una serie de tiempo estacionaria simplifica el análisis al permitir la aplicación de técnicas y modelos estadísticos más tradicionales. Por el contrario, si una serie de tiempo no es estacionaria, puede requerir la aplicación de técnicas de descomposición, transformación o modelado más sofisticadas para capturar la tendencia, la estacionalidad u otros patrones presentes en los datos.



###Test de Dickey-Fuller Aumentado (ADF)
Este test se utiliza para evaluar la estacionariedad de una serie de tiempo. La interpretación del resultado del test de Dickey-Fuller se basa en el valor del estadístico ADF y el valor p.

*   Por ejemplo un estadístico ADF = -3.689847 indica que la serie de tiempo
    tiene un valor más negativo que el valor crítico del 1%, del 5% y del 10%.
    Valores críticos:
   	1%: -3.438
	  5%: -2.865
	  10%: -2.569.
    Esto sugiere que existe evidencia estadística para rechazar la hipótesis nula de que la serie de tiempo tiene una raíz unitaria y, por lo tanto, no es estacionaria.
*   Un valor de valor p de 0.000144 también respalda esta conclusión. Un valor  
     p menor que un nivel de significancia predefinido (generalmente 0.05) indica que hay suficiente evidencia para rechazar la hipótesis nula de no estacionariedad.
*   En resumen, el resultado del test de Dickey-Fuller indica que la serie de
    tiempo es estacionaria, ya que el estadístico ADF es menor que los valores críticos y el valor p es significativamente menor que el nivel de significancia seleccionado. Esto implica que no hay una tendencia o estructura sistemática en la serie de tiempo y puede ser más adecuada para aplicar modelos y técnicas que asumen estacionariedad.

En este caso en base a los resultados de aplicar el test es necesario  transformar los datos  

In [6]:
np.random.seed(912231)
data2.sort_index(inplace=True)
data2.dropna(inplace=True)
for column in data2.columns:
    result = adfuller(data2[column])
    print(f'Variable: {column}')
    print('ADF Statistic: {:.6f}'.format(result[0]))
    print('p-value: {:.6f}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.4f}')
    print()

Variable: CPI
ADF Statistic: -0.576273
p-value: 0.876254
Critical Values:
   1%: -3.4532
   5%: -2.8716
   10%: -2.5721

Variable: Inflacion
ADF Statistic: -5.761090
p-value: 0.000001
Critical Values:
   1%: -3.4531
   5%: -2.8716
   10%: -2.5721

Variable: Expec. Inflacion
ADF Statistic: -5.511612
p-value: 0.000002
Critical Values:
   1%: -3.4525
   5%: -2.8713
   10%: -2.5720

Variable: Senti. Inflacion
ADF Statistic: -2.050575
p-value: 0.264847
Critical Values:
   1%: -3.4525
   5%: -2.8713
   10%: -2.5720

Variable: Indice Ind. Manuf
ADF Statistic: -2.416591
p-value: 0.137108
Critical Values:
   1%: -3.4528
   5%: -2.8714
   10%: -2.5720

Variable: Ventas Minoristas Mensuales
ADF Statistic: -18.705935
p-value: 0.000000
Critical Values:
   1%: -3.4523
   5%: -2.8712
   10%: -2.5719

Variable: PPI Mat. Construccion
ADF Statistic: -0.343182
p-value: 0.919162
Critical Values:
   1%: -3.4523
   5%: -2.8712
   10%: -2.5719

Variable: Tasa Interes
ADF Statistic: -1.717710
p-value: 0.42201

###Test de Dickey-Fuller Aumentado con Datos transformados
Se transformó la serie de tiempo: eliminando valores nulos y se aplicó una transformación diff(esta transformación calcula la diferencia entre observaciones consecutivas) para eliminar la tendencia o la estacionalidad `data2 = data2.diff().dropna()`

In [7]:
# Aplicar prueba de Dickey-Fuller aumentada (ADF)
np.random.seed(87051)
data2_ST = data2 # se mantiene una copia del dataframe sin transformación de datos
data2_ST.sort_index(inplace=True)
data2_ST.dropna(inplace=True)
data2['Tasa Interes'] = data2['Tasa Interes'].diff()
data2['CPI'] = data2['CPI'].diff()
data2['Senti. Inflacion'] = data2['Senti. Inflacion'].diff()
data2['Indice Ind. Manuf'] = data2['Indice Ind. Manuf'].diff()
data2['PPI Mat. Construccion'] = data2['PPI Mat. Construccion'].diff()
data2['Tasa Desempleo'] = data2['Tasa Desempleo'].diff()
data2.sort_index(inplace=True)
data2.dropna(inplace=True)

for column in data2.columns:
    result = adfuller(data2[column])
    print(f'Variable: {column}')
    print('ADF Statistic: {:.6f}'.format(result[0]))
    print('p-value: {:.6f}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.6f}')
    print()

Variable: CPI
ADF Statistic: -6.229084
p-value: 0.000000
Critical Values:
   1%: -3.453182
   5%: -2.871593
   10%: -2.572127

Variable: Inflacion
ADF Statistic: -5.724091
p-value: 0.000001
Critical Values:
   1%: -3.453182
   5%: -2.871593
   10%: -2.572127

Variable: Expec. Inflacion
ADF Statistic: -5.487665
p-value: 0.000002
Critical Values:
   1%: -3.452561
   5%: -2.871321
   10%: -2.571982

Variable: Senti. Inflacion
ADF Statistic: -12.330828
p-value: 0.000000
Critical Values:
   1%: -3.452486
   5%: -2.871288
   10%: -2.571964

Variable: Indice Ind. Manuf
ADF Statistic: -4.517270
p-value: 0.000183
Critical Values:
   1%: -3.452790
   5%: -2.871422
   10%: -2.572035

Variable: Ventas Minoristas Mensuales
ADF Statistic: -18.692378
p-value: 0.000000
Critical Values:
   1%: -3.452337
   5%: -2.871223
   10%: -2.571929

Variable: PPI Mat. Construccion
ADF Statistic: -9.002531
p-value: 0.000000
Critical Values:
   1%: -3.452337
   5%: -2.871223
   10%: -2.571929

Variable: Tasa Intere

In [93]:
data2.head(10)

,CPI,Inflacion,Expec. Inflacion,Senti. Inflacion,Indice Ind. Manuf,Ventas Minoristas Mensuales,PPI Mat. Construccion,Tasa Interes,Tasa Desempleo,Tasa USD Circulante
1992-04-01,0.3,0.215672,3.0,-6.0,-0.2769,0.8,-7.000000e-01,-0.11,-1.000000e-01,-0.126185
1992-05-01,0.3,0.215208,2.9,0.8,0.0109,0.7,-2.000000e-01,-0.17,0.000000e+00,-0.032321
1992-06-01,0.4,0.286328,3.1,-0.8,-0.1535,0.5,-1.421085e-14,0.34,2.000000e-01,-0.149899
1992-07-01,0.4,0.285510,2.7,-5.0,0.3613,0.6,-1.000000e-01,-0.15,8.881784e-16,0.005887
1992-08-01,0.3,0.213523,2.8,3.3,-0.8393,0.2,2.000000e-01,-0.45,-3.000000e-01,0.144228
1992-09-01,0.3,0.213068,3.0,0.0,0.3785,1.0,6.000000e-01,0.56,-8.881784e-16,0.343885
1992-10-01,0.6,0.425230,2.8,-1.8,0.2806,0.6,-1.000000e+00,-0.13,1.000000e-01,0.395431
1992-11-01,0.4,0.282287,2.9,14.3,-0.1385,0.0,5.000000e-01,-0.04,-3.000000e-01,0.075857
1992-12-01,0.2,0.140746,2.8,-6.3,-0.3013,1.3,9.000000e-01,0.11,4.000000e-01,-0.040815
1993-01-01,0.5,0.351370,2.9,-7.4,0.6244,1.2,5.000000e-01,-0.16,-1.000000e-01,-0.163327


In [9]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 301 entries, 1992-03-01 to 2017-03-01
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CPI                          301 non-null    float64
 1   Inflacion                    301 non-null    float64
 2   Expec. Inflacion             301 non-null    float64
 3   Senti. Inflacion             301 non-null    float64
 4   Indice Ind. Manuf            301 non-null    float64
 5   Ventas Minoristas Mensuales  301 non-null    float64
 6   PPI Mat. Construccion        301 non-null    float64
 7   Tasa Interes                 301 non-null    float64
 8   Tasa Desempleo               301 non-null    float64
 9   Tasa USD Circulante          301 non-null    float64
dtypes: float64(10)
memory usage: 25.9 KB


Luego de las transformaciones y de eliminar los valores Nan. La serie de tiempo queda con 274 observaciones. De 1954-10-01 a 2023-01-01

## Matriz de Correlación Lineal

In [10]:
import plotly.express as px

#matriz de correlación
corr_matrix = data2.corr()
column_names = corr_matrix.columns

# Crear un gráfico de calor utilizando Plotly
fig = px.imshow(corr_matrix.values,
                x=column_names,
                y=column_names,
                color_continuous_scale='Blues')

# Configurar etiquetas y título del gráfico
fig.update_layout(
    title='Matriz de Correlación',
   # xaxis_title='Variables',
   # yaxis_title='Variables'
)

# Mostrar los valores de los coeficientes en el gráfico
for i in range(len(column_names)):
    for j in range(len(column_names)):
        fig.add_annotation(
            x=column_names[j],
            y=column_names[i],
            text=str(round(corr_matrix.iloc[i, j], 2)),
            showarrow=False,
            font=dict(color='green', size=12)
        )

# Configurar la escala de colores en azul
fig.update_traces(colorbar=dict(title='Coeficiente', tickcolor='black', tickfont=dict(color='green')))

# Mostrar el gráfico
fig.show()


la salida de la matriz de correlación indica la relación de cada variable con la variable objetivo "Inflación". La columna "Inflación" muestra la correlación de cada variable con respecto a la variable objetivo, medida en valores absolutos. Los valores de correlación oscilan entre 0 y 1. Una correlación de 1 indica una correlación perfecta positiva, lo que significa que las variables están altamente relacionadas de manera positiva.

*   "CPI" (Índice de Precios al Consumidor) tiene una correlación de 0.49 con la variable "Inflación". Esto indica una correlación positiva moderada en el contexto de este análisis

*   "Tasa Desempleo" tiene una correlación de 0.25 con la variable "Inflación". Esta correlación es baja, lo que indica una relación débil entre la tasa de desempleo y la inflación. Es posible que otros factores tengan una influencia más significativa en la inflación.
* "PBI" (Producto Interno Bruto) tiene una correlación de 0.17 con la variable "Inflación". Al igual que la tasa de desempleo, esta correlación es baja, lo que sugiere que el PBI no está fuertemente relacionado con la inflación en este contexto.
* "Tasa Interés" tiene una correlación de 0.23 con la variable "Inflación". Esta correlación también es baja, lo que indica que la tasa de interés no tiene una relación fuerte con la inflación en el contexto de este análisis


In [46]:
#Correlación con variable Target "Inflacion"
target = "Inflacion"
corr_matrix[target].apply(lambda x: abs(x)).sort_values(ascending=False)


Inflacion                      1.000000
CPI                            0.988742
Ventas Minoristas Mensuales    0.351297
Expec. Inflacion               0.276622
PPI Mat. Construccion          0.136336
Tasa USD Circulante            0.135950
Senti. Inflacion               0.103046
Indice Ind. Manuf              0.101470
Tasa Interes                   0.098502
Tasa Desempleo                 0.018963
Name: Inflacion, dtype: float64

# Paso 3: Generar Conjuntos de Datos de Entrenamiento y Test

In [11]:
np.random.seed(1123)
data2.sort_index(inplace=True)
data2_ST.sort_index(inplace=True)

# Crear variable Target (predicción)
y = data2['Inflacion']
x = data2.drop(['Inflacion'], axis=1).loc[y.index]

y1 = data2_ST['Inflacion']
x1 = data2_ST.drop(['Inflacion'], axis=1).loc[y1.index]
# Dividir
#train_size = int(len(y) * 0.8)
#x_train, y_train = x.iloc[:train_size,:], y.iloc[:train_size]
#x_test, y_test = x.iloc[train_size:,:], y.iloc[train_size:]
#random_state = 42

# Crear el objeto TimeSeriesSplit para la validación cruzada en serie temporal
tscv = TimeSeriesSplit(n_splits=3)

# Dividir los datos en conjuntos de entrenamiento y prueba
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state)

for train_index, test_index in tscv.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    x1_train, x1_test = x1.iloc[train_index], x1.iloc[test_index]
    y1_train, y1_test = y1.iloc[train_index], y1.iloc[test_index]

# Paso 4: Funciones de Predicción

In [12]:
np.random.seed(41235)

## RAMDON FOREST
def train_rf(x_train, y_train, x_test):
    # Entrenar modelo Random Forest
    modeloRF = RandomForestRegressor(n_estimators=100, random_state=0)
    modeloRF.fit(x_train, y_train)
    # Realizar predicciones
    y_pred = modeloRF.predict(x_test)
    with open('modeloRF.pickle', 'wb') as f:
       pickle.dump(modeloRF, f)
    return y_pred
## XGBOOST
def train_xgboost(x_train, y_train, x_test):
    # Entrenar modelo XGBoost
    modeloXGB = XGBRegressor(n_estimators=100, random_state=0)
    modeloXGB.fit(x_train, y_train)
    # Realizar predicciones
    y_pred = modeloXGB.predict(x_test)
    with open('modeloXGB.pickle', 'wb') as f:
        pickle.dump(modeloXGB, f)
    return y_pred
## Linear Regression
def train_LinearRegression(x_train, y_train, x_test):
    #Entrenar Modelo de Regresion Lineal
    modeloLR = LinearRegression()
    modeloLR.fit(x_train,y_train)
    # Realizar predicciones
    y_pred = modeloLR.predict(x_test)
    with open('modeloLR.pickle', 'wb') as f:
        pickle.dump(modeloLR, f)

    return y_pred


In [13]:
np.random.seed(412352)

## RAMDON FOREST
def train_rf(x1_train, y1_train, x1_test):
    # Entrenar modelo Random Forest
    modeloRF = RandomForestRegressor(n_estimators=100, random_state=0)
    modeloRF.fit(x1_train, y1_train)
    # Realizar predicciones
    y1_pred = modeloRF.predict(x1_test)
    with open('modeloRF_ST.pickle', 'wb') as f:
       pickle.dump(modeloRF, f)
    return y1_pred
## XGBOOST
def train_xgboost(x1_train, y1_train, x1_test):
    # Entrenar modelo XGBoost
    modeloXGB = XGBRegressor(n_estimators=100, random_state=0)
    modeloXGB.fit(x1_train, y1_train)
    # Realizar predicciones
    y1_pred = modeloXGB.predict(x1_test)
    with open('modeloXGB_ST.pickle', 'wb') as f:
        pickle.dump(modeloXGB, f)
    return y1_pred
## Linear Regression
def train_LinearRegression(x1_train, y1_train, x1_test):
    #Entrenar Modelo de Regresion Lineal
    modeloLR = LinearRegression()
    modeloLR.fit(x1_train,y1_train)
    # Realizar predicciones
    y1_pred = modeloLR.predict(x1_test)
    with open('modeloLR_ST.pickle', 'wb') as f:
        pickle.dump(modeloLR, f)

    return y1_pred

# Paso 5: Métricas
Las métricas en la evaluación de modelos de ML nos permiten medir la precisión, el error y la capacidad de explicación del modelo. Al utilizar estas métricas en conjunto, se puede obtener una imagen completa del rendimiento del modelo y tomar decisiones sobre su utilidad y eficacia.

## MAE (Mean Absolute Error) Error Absoluto Medio:
• Propósito: El MAE mide el promedio de las diferencias absolutas entre los valores predichos y los valores reales.
•	Cálculo: Se calcula sumando las diferencias absolutas y dividiendo por el número total de observaciones.
•	Uso: El MAE también se utiliza para evaluar la precisión de un modelo de regresión. A diferencia del MSE, el MAE no penaliza de manera desproporcionada los errores grandes, lo que lo hace más útil cuando se desea evaluar el rendimiento de un modelo sin importar la magnitud de los errores.
•	Comparación: Al igual que el MSE, un valor de MAE más bajo indica un mejor ajuste del modelo. Al comparar modelos, se prefiere aquellos con un MAE más bajo.
•	Interpretación: El MAE tiene una interpretación intuitiva y directa, ya que está en las mismas unidades que los datos originales. Por ejemplo, en una predicción de tasa de inflación si el MAE es 0.5, significa que, en promedio, la predicción difiere en 0.5 % de la inflación real.

## MSE (Mean Square Error) Error Cuadrático Medio
El MSE se calcula como el promedio de los cuadrados de las diferencias entre los valores predichos y los valores reales. Un MSE más bajo indica que las predicciones del modelo son más cercanas a los valores reales en promedio. Se considera que un MSE más bajo indica una mejor precisión en las predicciones debido a que:
•	El MSE penaliza más los errores grandes. Al elevar al cuadrado las diferencias, los errores más grandes tienen un impacto mayor en el resultado final. Esto significa que si el modelo comete algunos errores significativos, el MSE aumentará rápidamente. Por lo tanto, un MSE más bajo indica que el modelo está cometiendo en promedio menos errores grandes.

## R^2  Coeficiente de Determinación
Esta métrica se utiliza para evaluar la calidad de un modelo de regresión. Es una medida de qué tan bien se ajustan los valores predichos por el modelo a los valores
El R^2 se interpreta de la siguiente manera:
*   Un valor cercano a 1 indica que el modelo explica una gran parte de la variabilidad de los datos y se ajusta bien a los datos.
*   Un valor cercano a 0 indica que el modelo no explica mucha variabilidad y no se ajusta bien a los datos.
*   Es importante tener en cuenta que un valor negativo de R^2 no es una situación común y, en general, si se obtiene un valor negativo, es una señal de que el modelo no es apropiado para los datos y debe ser revisado.




In [14]:
np.random.seed(12335)

#RANDOM FOREST OBSERVADO/MSE/MAE
rf_pred = train_rf(x_train, y_train, x_test)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

#XGBOOST OBSERVADO/MSE/MAE
xgb_pred = train_xgboost(x_train, y_train, x_test)
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

#LINEAR REGRESSION OBSERVADO/MSE/MAE
lr_pred = train_LinearRegression(x_train, y_train, x_test)
lr_mse = mean_squared_error(y_test,lr_pred)
lr_mae = mean_absolute_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)


print('Imprimir métricas')

print('Random Forest:')
print(f'MSE: {rf_mse:.2f}')
print(f'MAE: {rf_mae:.2f}')
print(f'R2: {rf_r2:.2f}\n')


print('Linear Regression:')
print(f'MSE: {lr_mse:.2f}')
print(f'MAE: {lr_mae:.2f}')
print(f'R2: {lr_r2:.2f}\n')
print('XGBoost:')
print(f'MSE: {xgb_mse:.2f}')
print(f'MAE: {xgb_mae:.2f}')
print(f'R2: {xgb_r2:.2f}\n')

Imprimir métricas
Random Forest:
MSE: 0.00
MAE: 0.05
R2: 0.92

Linear Regression:
MSE: 0.00
MAE: 0.05
R2: 0.93

XGBoost:
MSE: 0.00
MAE: 0.05
R2: 0.90



In [32]:
np.random.seed(12335)
# Datos originales sin transformar
#RANDOM FOREST OBSERVADO/MSE/MAE
rf_pred1 = train_rf(x1_train, y1_train, x1_test)
rf_mse = mean_squared_error(y1_test, rf_pred1)
rf_mae = mean_absolute_error(y1_test, rf_pred1)
rf_r2 = r2_score(y1_test, rf_pred1)

#XGBOOST OBSERVADO/MSE/MAE
xgb_pred1 = train_xgboost(x1_train, y1_train, x1_test)
xgb_mse = mean_squared_error(y1_test, xgb_pred1)
xgb_mae = mean_absolute_error(y1_test, xgb_pred1)
xgb_r2 = r2_score(y1_test, xgb_pred1)

#LINEAR REGRESSION OBSERVADO/MSE/MAE
lr_pred1 = train_LinearRegression(x1_train, y1_train, x1_test)
lr_mse = mean_squared_error(y1_test,lr_pred1)
lr_mae = mean_absolute_error(y1_test, lr_pred1)
lr_r2 = r2_score(y1_test, lr_pred1)


print('Imprimir métricas Variables sin Transformar')

print('Random Forest:')
print(f'MSE: {rf_mse:.2f}')
print(f'MAE: {rf_mae:.2f}')
print(f'R2: {rf_r2:.2f}\n')


print('Linear Regression:')
print(f'MSE: {lr_mse:.2f}')
print(f'MAE: {lr_mae:.2f}')
print(f'R2: {lr_r2:.2f}\n')
print('XGBoost:')
print(f'MSE: {xgb_mse:.2f}')
print(f'MAE: {xgb_mae:.2f}')
print(f'R2: {xgb_r2:.2f}\n')

Imprimir métricas Variables sin Transformar
Random Forest:
MSE: 0.00
MAE: 0.05
R2: 0.92

Linear Regression:
MSE: 0.00
MAE: 0.05
R2: 0.93

XGBoost:
MSE: 0.00
MAE: 0.05
R2: 0.90



In [16]:
df_modelos = pd.DataFrame({'Observado': y_test, 'XGB': xgb_pred, 'RForest': rf_pred, 'LR': lr_pred})
df_modelos.index = pd.to_datetime(df_modelos.index)
df_modelos = df_modelos.sort_index()
fig = go.Figure()

fig.add_trace(go.Scatter(y=df_modelos['Observado']/100, x=df_modelos.index, mode='lines', name='Observado'))
fig.add_trace(go.Scatter(y=df_modelos['XGB']/100, x=df_modelos.index, mode='lines', name='XGBoost'))
fig.add_trace(go.Scatter(y=df_modelos['RForest']/100, x=df_modelos.index, mode='lines', name='Random Forest'))
fig.add_trace(go.Scatter(y=df_modelos['LR']/100, x=df_modelos.index, mode='lines', name='Linear Regression'))


fig.update_layout(title= 'Predicciones de inflación en Estados Unidos', showlegend=True ,
                  xaxis=dict(
                      title= 'Período'
                  ),
                  yaxis=dict(
                      tickformat=".2%",
                      title='Tasa de Inflación'
                      ))
# Mostrar plot
fig.show()

In [34]:
df_modelosST = pd.DataFrame({'Observado': y1_test, 'XGB': xgb_pred1, 'RForest': rf_pred1, 'LR': lr_pred1})
df_modelosST.index = pd.to_datetime(df_modelosST.index)
df_modelosST = df_modelosST.sort_index()
fig = go.Figure()

fig.add_trace(go.Scatter(y=df_modelosST['Observado']/100, x=df_modelos.index, mode='lines', name='Observado'))
fig.add_trace(go.Scatter(y=df_modelosST['XGB']/100, x=df_modelos.index, mode='lines', name='XGBoost'))
fig.add_trace(go.Scatter(y=df_modelosST['RForest']/100, x=df_modelos.index, mode='lines', name='Random Forest'))
fig.add_trace(go.Scatter(y=df_modelosST['LR']/100, x=df_modelos.index, mode='lines', name='Linear Regression'))


fig.update_layout(title= 'Predicciones de inflación en Estados Unidos - Datos Originales', showlegend=True ,
                  xaxis=dict(
                      title= 'Período'
                  ),
                  yaxis=dict(
                      tickformat=".2%",
                      title='Tasa de Inflación'
                      ))
# Mostrar plot
fig.show()

#Conclusiones


In [72]:

# Filtrar datos de un año específico
año = '2017'
df_modelosfil = df_modelos.loc[año]
print(df_modelosfil)

            Observado       XGB   RForest        LR
2017-01-01   0.404308  0.576308  0.566550  0.507444
2017-02-01   0.159266  0.228519  0.244843  0.224280
2017-03-01  -0.046720 -0.046678 -0.053532 -0.026535


In [20]:
num_filas = df_modelos.shape[0]

# Seleccionar todas las filas excepto las últimas 3
df_modelos = df_modelos.iloc[:num_filas-3]
df_modelosST = df_modelosST.iloc[:num_filas-3]

# Imprimir el DataFrame resultante
print(df_modelos)

            Observado       XGB   RForest        LR
2011-01-01   0.324304  0.339374  0.349772  0.396498
2011-02-01   0.321447  0.353574  0.363361  0.368163
2011-03-01   0.517355  0.602238  0.619933  0.596549
2011-04-01   0.469410  0.505891  0.550119  0.531359
2011-05-01   0.318171  0.372210  0.358895  0.357868
...               ...       ...       ...       ...
2016-08-01   0.184922  0.227507  0.227237  0.249017
2016-09-01   0.262321  0.337073  0.348497  0.340256
2016-10-01   0.234269  0.261036  0.288247  0.313009
2016-11-01   0.117895  0.164185  0.170008  0.170371
2016-12-01   0.252452  0.359189  0.349228  0.334420

[72 rows x 4 columns]


In [23]:
df_modelos['pred_absoluta_XGB'] = df_modelos['XGB'] + df_modelos['XGB'].apply(np.cumsum)
df_modelos['pred_absoluta_LR'] = df_modelos['LR'] + df_modelos['LR'].apply(np.cumsum)
df_modelos['pred_absoluta_RF'] = df_modelos['RForest'] + df_modelos['RForest'].apply(np.cumsum)

In [83]:
df_modelos.tail(10)

,Observado,XGB,RForest,LR,pred_absoluta_XGB,pred_absoluta_LR,pred_absoluta_RF
2016-03-01,0.313480,0.357655,0.360596,0.392630,[0.7153099775314331],[0.7852603289305304],[0.7211926575497304]
2016-04-01,0.383065,0.500342,0.498796,0.475699,[1.0006849765777588],[0.9513981994228353],[0.9975923109577327]
2016-05-01,0.236410,0.264915,0.272042,0.301074,[0.5298299193382263],[0.6021485043587782],[0.5440845981502099]
2016-06-01,0.277596,0.321917,0.346894,0.359024,[0.6438341736793518],[0.7180489092628679],[0.693787001231888]
2016-07-01,-0.050370,-0.053477,-0.052603,-0.031550,[-0.10695468634366989],[-0.06309935354222018],[-0.10520504285338499]
2016-08-01,0.184922,0.217202,0.232799,0.244309,[0.43440401554107666],[0.488618384941239],[0.4655984906979451]
2016-09-01,0.262321,0.350118,0.365616,0.341269,[0.7002361416816711],[0.6825383884454088],[0.7312328719516951]
2016-10-01,0.234269,0.263167,0.271891,0.311874,[0.5263347625732422],[0.6237476722364392],[0.5437824677417619]
2016-11-01,0.117895,0.167068,0.162366,0.162529,[0.33413684368133545],[0.3250583561621424],[0.3247325499407019]
2016-12-01,0.252452,0.332881,0.350500,0.332477,[0.6657625436782837],[0.6649537350972596],[0.700999218311212]


In [24]:
df_modelos.index = pd.to_datetime(df_modelos.index)
df_modelos = df_modelos.sort_index()
fig = go.Figure()
df_modelos['pred_absoluta_RF'] = df_modelos['pred_absoluta_RF'].apply(lambda x: x[0])
#df_modelos['pred_absoluta_XGB'] = df_modelos['pred_absoluta_XGB'].apply(lambda x: x[0])
fig.add_trace(go.Scatter(y=df_modelos['Observado']/100, x=df_modelos.index, mode='lines', name='Observado'))
fig.add_trace(go.Scatter(y=df_modelos['pred_absoluta_RF']/100, x=df_modelos.index, mode='lines', name='Random Forest'))
#fig.add_trace(go.Scatter(y=df_modelos['pred_absoluta_XGB']/100, x=df_modelos.index, mode='lines', name='XGBoost'))

fig.update_layout(title= 'Predicciones de Inflación en Estados Unidos - Valores Absolutos ', showlegend=True ,
                  xaxis=dict(
                      title= 'Período'
                  ),
                  yaxis=dict(
                      tickformat=".2%",
                      title='Tasa de Inflación'
                      ))
# Mostrar plot
fig.show()

In [21]:
df_modelosST

,Observado,XGB,RForest,LR
2011-01-01,0.324304,0.339374,0.349772,0.396498
2011-02-01,0.321447,0.353574,0.363361,0.368163
2011-03-01,0.517355,0.602238,0.619933,0.596549
2011-04-01,0.469410,0.505891,0.550119,0.531359
2011-05-01,0.318171,0.372210,0.358895,0.357868
...,...,...,...,...
2016-08-01,0.184922,0.227507,0.227237,0.249017
2016-09-01,0.262321,0.337073,0.348497,0.340256
2016-10-01,0.234269,0.261036,0.288247,0.313009
2016-11-01,0.117895,0.164185,0.170008,0.170371


In [35]:
df_modelosST.index = pd.to_datetime(df_modelosST.index)
df_modelosST = df_modelosST.sort_index()
fig = go.Figure()
#df_modelosST['pred_absoluta_RF'] = df_modelosST['pred_absoluta_RF'].apply(lambda x: x[0])
#df_modelos['pred_absoluta_XGB'] = df_modelos['pred_absoluta_XGB'].apply(lambda x: x[0])
fig.add_trace(go.Scatter(y=df_modelosST['Observado']/100, x=df_modelosST.index, mode='lines', name='Observado'))
fig.add_trace(go.Scatter(y=df_modelosST['RForest']/100, x=df_modelosST.index, mode='lines', name='Random Forest'))
#fig.add_trace(go.Scatter(y=df_modelos['pred_absoluta_XGB']/100, x=df_modelos.index, mode='lines', name='XGBoost'))

fig.update_layout(title= 'Predicciones de Inflación en Estados Unidos - Valores Absolutos ', showlegend=True ,
                  xaxis=dict(
                      title= 'Período'
                  ),
                  yaxis=dict(
                      tickformat=".2%",
                      title='Tasa de Inflación'
                      ))
# Mostrar plot
fig.show()

In [85]:
df_modelos['pred_absoluta_RF']

2011-01-01    0.691951
2011-02-01    0.736209
2011-03-01    1.193615
2011-04-01    1.120097
2011-05-01    0.736514
                ...   
2016-08-01    0.465598
2016-09-01    0.731233
2016-10-01    0.543782
2016-11-01    0.324733
2016-12-01    0.700999
Name: pred_absoluta_RF, Length: 72, dtype: float64

In [90]:
df_modelos['Residuos'] = df_modelos['Observado'] - df_modelos['pred_absoluta_RF']
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_modelos.index,
    y=df_modelos['Residuos'],
    mode='markers',
    name='Residuos'
  ))

# Línea horizontal en cero
fig.add_shape(
    type='line',
    x0=df_modelos.index[0],
    y0=0,
    x1=df_modelos.index[-1],
    y1=0,
    line=dict(
        color='red',
        width=2,
        dash='dash'
    )
)

# Establecer título y etiquetas de los ejes
fig.update_layout(
    title='Gráfico de Residuos',
    xaxis_title='Observaciones',
    yaxis_title='Diferencia (Residuos)'
)

# Mostrar el gráfico
fig.show()

In [88]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_modelos['Observado'], y=df_modelos['pred_absoluta_RF'], mode='markers', name='Valores Predichos'))

# Agregar una línea diagonal para una comparación visual
fig.add_trace(go.Scatter(x=[min(df_modelos['Observado']), max(df_modelos['Observado'])], y=[min(df_modelos['Observado']), max(df_modelos['Observado'])],
                         mode='lines', name='Línea Diagonal', line=dict(color='red', dash='dash')))

# Diseño del gráfico
fig.update_layout(
    xaxis_title='Valores Reales',
    yaxis_title='Valores Predichos',
    title='Gráfico de Dispersión: Valores Reales vs. Valores Predichos'
)

# Mostrar el gráfico
fig.show()

In [27]:
resultados = df_modelos[['Observado','pred_absoluta_RF']]
resultados.to_csv('resultadosRF1ultimo.csv', index_label='Date')

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_modelosST['Observado'], y=df_modelosST['RForest'], mode='markers', name='Valores Predichos'))

# Agregar una línea diagonal para una comparación visual
fig.add_trace(go.Scatter(x=[min(df_modelosST['Observado']), max(df_modelosST['Observado'])], y=[min(df_modelosST['Observado']), max(df_modelosST['Observado'])],
                         mode='lines', name='Línea Diagonal', line=dict(color='red', dash='dash')))

# Diseño del gráfico
fig.update_layout(
    xaxis_title='Valores Reales',
    yaxis_title='Valores Predichos',
    title='Gráfico de Dispersión: Valores Reales vs. Valores Predichos'
)

# Mostrar el gráfico
fig.show()

In [28]:
resultados = df_modelosST[['Observado','RForest']]
resultados.to_csv('resultadosRF1ultimoST.csv', index_label='Date')

In [29]:
df_modelosST

,Observado,XGB,RForest,LR
2011-01-01,0.324304,0.339374,0.349772,0.396498
2011-02-01,0.321447,0.353574,0.363361,0.368163
2011-03-01,0.517355,0.602238,0.619933,0.596549
2011-04-01,0.469410,0.505891,0.550119,0.531359
2011-05-01,0.318171,0.372210,0.358895,0.357868
...,...,...,...,...
2016-08-01,0.184922,0.227507,0.227237,0.249017
2016-09-01,0.262321,0.337073,0.348497,0.340256
2016-10-01,0.234269,0.261036,0.288247,0.313009
2016-11-01,0.117895,0.164185,0.170008,0.170371
